# Running the prompts against several models for code quality comparison

Models:
- GPT 4.1
- GPT 4o
- GPT o3
- GPT o1
- Claude Sonnet 3.7
- Claude Sonnet 4.0
- Deepseek

Annotating the prompts by characteristics:
- Personification score
- request type (question vs command)
- length?
-

In [1]:
import sqlite3
import pandas as pd
import os

db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
prompts = pd.read_sql("SELECT * FROM translated_scratch_prompts", conn)
prompts = prompts.drop(columns=['conversational', 'code', 'other'])
prompts

,conversation_id,message_id,role,message_text,gender,user_id,language
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,en
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,en
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,en
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,en
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,en
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,en
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,en
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,de
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,de
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,en


## Open AI & Deepseek

In [2]:
from openai import OpenAI

gpt_client = OpenAI()
#"o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06"
gpt_models = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06"]

deepseek_client = OpenAI(api_key=os.environ.get("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")
deepseek_models = ["deepseek-chat"]

def prompt_gpt(client, model, prompt):
    response = client.responses.create(
        model=model,
        input=prompt
    )
    return response.output_text

for model in gpt_models:
    prompts[model] = prompts["message_text"].apply(lambda x: prompt_gpt(gpt_client, model, x))

def prompt_deepseek(client, model, prompt):
    response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            #max_tokens=1000,
            #temperature=0.0
        )

    return response.choices[0].message.content


# for model in deepseek_models:
#     prompts[model] = prompts["message_text"].apply(lambda x: prompt_deepseek(deepseek_client, model, x))

prompts



,conversation_id,message_id,role,message_text,gender,user_id,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,gpt-4.1-2025-04-14,gpt-4o-2024-08-06
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,en,To perform **dummy hot encoding** (also called...,With scikit-learn the usual way to create dumm...,Great question! You can use **`OneHotEncoder`*...,You can use **`OneHotEncoder`** from `sklearn....,To apply Dummy Hot encoding using `sklearn` fo...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,en,To encode and compress a JavaScript object int...,There isn’t a single “best” technique that bea...,Great question! You want to take a JavaScript ...,Encoding and compressing a JavaScript object i...,To encode and compress a JavaScript object int...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,en,Certainly! You can split the `persona` column ...,You can treat each value in the persona column...,Yes! You can split the `persona` column into m...,Absolutely! You want to **split the informatio...,To separate the `persona` column into multiple...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,en,"That's a great productivity idea, especially f...",Your idea is perfectly feasible and—if you col...,"That’s a great idea — this kind of ""research p...",Your idea is **very practical and quite useful...,Your idea is practical and can save a lot of t...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,en,You can achieve this in **Blender (3.x or newe...,Below is a standalone script that you can past...,Got it. You want a Blender Python script that ...,Certainly! Below is a script for **Blender** (...,"To achieve this in Blender using Python, you c..."
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,en,"To run a Python coroutine or future ""non-block...",`await` itself is always “all or nothing”: if ...,You're basically describing **driving a Python...,To **run a Python `async` function (coroutine)...,To run a Python `Future` or coroutine such tha...
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,en,Certainly! Here's a short Python script you ca...,```python\nimport pandas as pd\nimport re # ...,"Got it! You want to load your CSV, look at the...",Certainly! Here's a short Python script for yo...,Certainly! Here's a short Python script to rea...
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,de,To combine two GRIB files and save the result ...,"Below is a practical, step-by-step recipe you ...",Great question! Combining **GRIB** files often...,Sure! Combining two **GRIB** files in Python (...,To combine two GRIB files in a Jupyter Noteboo...
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,de,To help you read and parse the contents of an ...,Sure—let me know exactly what you need to do w...,Got it! You're working with Python and need to...,Sure! NMEA files typically contain GPS data in...,"To read an NMEA file in Python, you'll typical..."
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,en,Sure! Here's a Python method to unzip a `.zip`...,"Below is a compact, reusable Python function t...",Sure! You can use Python's built-in `zipfile` ...,Certainly! You can unzip files in Python using...,"To unzip a file in Python, you can use the `zi..."


In [26]:
prompts

,conversation_id,message_id,role,message_text,gender,user_id,language,chatgpt-4o-latest,o3-2025-04-16
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,en,To perform **dummy hot encoding** (a variation...,With scikit-learn you use the class sklearn.pr...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,en,To encode and compress a **JavaScript object**...,Below is a proven pattern that balances simpli...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,en,Certainly! You can extract the information fro...,Yes. \nTreat every line break as the separato...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,en,That's a **great and practical idea**—and you'...,Your idea is perfectly feasible and—judging fr...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,en,To replicate a collection of objects in **Blen...,Below is a self-contained script you can paste...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,en,To run a Python `Future` or coroutine *without...,In `asyncio` you don’t need any special API to...
6,15,242,user,hey can you write me a short python script for...,Woman (cisgender),48,en,Sure! Below is a short Python script you can u...,"Here’s a minimal, self-contained cell you can ..."
7,18,266,user,how can I combine two grib files in jupyter no...,Woman (cisgender),60,de,To combine two GRIB files in a Jupyter Noteboo...,Below are three practical ways you can “combin...
8,20,268,user,I work with Python and have to read an NMEA fi...,Woman (cisgender),65,de,Great! NMEA (National Marine Electronics Assoc...,Sure – could you upload or paste a short porti...
9,21,290,user,please write method to unzip file in python,Woman (cisgender),73,en,Sure! Here's a simple method in Python to unzi...,"Below is a compact, production-ready helper th..."


## Anthropic

In [7]:
from anthropic import Anthropic
import dotenv
dotenv.load_dotenv()

models = [ "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]
claude_client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

def prompt_claude(client, model, prompt):
    response = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=model,
    )
    return "".join(block.text for block in response.content if hasattr(block, "text"))

for model in models:
    prompts[model] = prompts["message_text"].apply(lambda x: prompt_claude(claude_client, model, x))

In [8]:
prompts

,conversation_id,message_id,role,message_text,gender,user_id,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,gpt-4.1-2025-04-14,gpt-4o-2024-08-06,claude-3-7-sonnet-20250219,claude-sonnet-4-20250514,code_chatgpt-4o-latest,code_o3-2025-04-16,code_gpt-5-chat-latest,code_gpt-4.1-2025-04-14,code_gpt-4o-2024-08-06
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,en,To perform **dummy hot encoding** (also called...,With scikit-learn the usual way to create dumm...,Great question! You can use **`OneHotEncoder`*...,You can use **`OneHotEncoder`** from `sklearn....,To apply Dummy Hot encoding using `sklearn` fo...,# Using One-Hot Encoding for 'embarked' Column...,You can use sklearn's `OneHotEncoder` to perfo...,[import pandas as pd\n\n# Sample DataFrame\ndf...,[import pandas as pd\nfrom sklearn.preprocessi...,[import pandas as pd\nfrom sklearn.preprocessi...,[from sklearn.preprocessing import OneHotEncod...,[import pandas as pd\n from sklearn.preproce...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,en,To encode and compress a JavaScript object int...,There isn’t a single “best” technique that bea...,Great question! You want to take a JavaScript ...,Encoding and compressing a JavaScript object i...,To encode and compress a JavaScript object int...,# Encoding JavaScript Objects for URLs\n\nWhen...,Here are several effective approaches to encod...,[],[],[],[],[]
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,en,Certainly! You can split the `persona` column ...,You can treat each value in the persona column...,Yes! You can split the `persona` column into m...,Absolutely! You want to **split the informatio...,To separate the `persona` column into multiple...,# Parsing Persona Data in Pandas DataFrame\n\n...,I'll help you extract the persona column and c...,[import pandas as pd\n\n# Sample DataFrame (re...,[import pandas as pd\n\n# starting dataframe -...,[import pandas as pd\n\n# Example DataFrame\nd...,[import pandas as pd\n\n# Example DataFrame\nd...,[import pandas as pd\n\n# Sample DataFrame\nda...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,en,"That's a great productivity idea, especially f...",Your idea is perfectly feasible and—if you col...,"That’s a great idea — this kind of ""research p...",Your idea is **very practical and quite useful...,Your idea is practical and can save a lot of t...,# Evaluation of PDF Auto-Renaming Project for ...,Your idea is excellent and very practical! Thi...,[import os\n\ndef list_pdfs(directory):\n r...,[],[from PyPDF2 import PdfReader\n\nreader = PdfR...,[import os\nfrom PyPDF2 import PdfReader\n\nde...,[import os\nimport PyPDF2\n\ndef extract_title...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,en,You can achieve this in **Blender (3.x or newe...,Below is a standalone script that you can past...,Got it. You want a Blender Python script that ...,Certainly! Below is a script for **Blender** (...,"To achieve this in Blender using Python, you c...",# Blender Script: Convert Meshes to Cube Insta...,Here's a Python script for Blender that will a...,[import bpy\nimport mathutils\n\n# === CONFIGU...,[import bpy\nimport mathutils\n\n# -----------...,[import bpy\nimport mathutils\n\ndef create_cu...,[import bpy\nimport mathutils\n\nSOURCE_COLLEC...,[import bpy\nimport mathutils\n\n# Get the ori...
5,13,126,user,"how to run a Python future without blocking, i...",Man (cisgender),46,en,"To run a Python coroutine or future ""non-block...",`await` itself is always “all or nothing”: if ...,You're basically describing **driving a Python...,To **run a Python `async` function (coroutine)...,To run a Python `Future` or coroutine such tha...,# Running Python Futures Without Blocking\n\nT...,You're describing the concept of **stepping th...,[import asyncio\nfrom types import CoroutineTy...,[import asyncio\n\nasync def job():\n print...

## Parse Code Snippets

In [25]:
import re

def parse_code_snippets(text):
    """
    Extract code snippets marked with triple backticks and any of these language tags:
    python, javascript, js, typescript, ts.
    Joins all found code into a single string separated by newlines.
    """
    pattern = r"```(?:python|javascript|js|typescript|ts|html)[ \t]*\n(.*?)(?=```)"
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    code_snippets = [m.strip() for m in matches]
    return "\n".join(code_snippets)


columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]

for column in columns:
    prompts[f"code_{column}"] = prompts[column].apply(parse_code_snippets)

prompts

,conversation_id,message_id,role,message_text,gender,user_id,language,chatgpt-4o-latest,o3-2025-04-16,gpt-5-chat-latest,...,gpt-4o-2024-08-06,claude-3-7-sonnet-20250219,claude-sonnet-4-20250514,code_chatgpt-4o-latest,code_o3-2025-04-16,code_gpt-5-chat-latest,code_gpt-4.1-2025-04-14,code_gpt-4o-2024-08-06,code_claude-3-7-sonnet-20250219,code_claude-sonnet-4-20250514
0,6,5,user,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,en,To perform **dummy hot encoding** (also called...,With scikit-learn the usual way to create dumm...,Great question! You can use **`OneHotEncoder`*...,...,To apply Dummy Hot encoding using `sklearn` fo...,# Using One-Hot Encoding for 'embarked' Column...,You can use sklearn's `OneHotEncoder` to perfo...,import pandas as pd\n\n# Sample DataFrame\ndf ...,import pandas as pd\nfrom sklearn.preprocessin...,import pandas as pd\nfrom sklearn.preprocessin...,from sklearn.preprocessing import OneHotEncode...,import pandas as pd\n from sklearn.preproces...,from sklearn.preprocessing import OneHotEncode...,import pandas as pd\nfrom sklearn.preprocessin...
1,7,43,user,whats the best way to encode and compress a ja...,Man (cisgender),25,en,To encode and compress a JavaScript object int...,There isn’t a single “best” technique that bea...,Great question! You want to take a JavaScript ...,...,To encode and compress a JavaScript object int...,# Encoding JavaScript Objects for URLs\n\nWhen...,Here are several effective approaches to encod...,"import { compressToEncodedURIComponent, decomp...","<script src=""https://cdn.jsdelivr.net/npm/lz-s...","import LZString from ""lz-string"";\n\n// Exampl...",const jsonString = JSON.stringify(yourObject);...,const jsonString = JSON.stringify(yourObject);...,// Encoding\nfunction encodeObject(obj) {\n r...,// Encode\nfunction encodeObject(obj) {\n con...
2,8,47,user,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,en,Certainly! You can split the `persona` column ...,You can treat each value in the persona column...,Yes! You can split the `persona` column into m...,...,To separate the `persona` column into multiple...,# Parsing Persona Data in Pandas DataFrame\n\n...,I'll help you extract the persona column and c...,import pandas as pd\n\n# Sample DataFrame (rep...,import pandas as pd\n\n# starting dataframe --...,import pandas as pd\n\n# Example DataFrame\nda...,import pandas as pd\n\n# Example DataFrame\ndf...,import pandas as pd\n\n# Sample DataFrame\ndat...,import pandas as pd\nimport re\n\n# Function t...,import pandas as pd\nimport re\n\ndef parse_pe...
3,10,57,user,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,en,"That's a great productivity idea, especially f...",Your idea is perfectly feasible and—if you col...,"That’s a great idea — this kind of ""research p...",...,Your idea is practical and can save a lot of t...,# Evaluation of PDF Auto-Renaming Project for ...,Your idea is excellent and very practical! Thi...,import os\n\ndef list_pdfs(directory):\n re...,,from PyPDF2 import PdfReader\n\nreader = PdfRe...,import os\nfrom PyPDF2 import PdfReader\n\ndef...,import os\nimport PyPDF2\n\ndef extract_title(...,"def scan_directory(directory_path):\n """"""Sc...",import os\nimport re\nimport fitz # PyMuPDF\n...
4,12,65,user,Blender and Python. I have a collection of hun...,Man (cisgender),34,en,You can achieve this in **Blender (3.x or newe...,Below is a standalone script that you can past...,Got it. You want a Blender Python script that ...,...,"To achieve this in Blender using Python, you c...",# Blender Script: Convert Meshes to Cube Insta...,Here's a Python script for Blender that will a...,import bpy\nimport mathutils\n\n# === CONFIGUR...,import bpy\nimport mathutils\n\n# ------------...,import bpy\nimport mathutils\n\ndef create_cub...,import bpy\nimport mathutils\n\nSOURCE_COLLECT...,import bpy\nimport mathutils\n\n# Get the orig...,import bpy\nimport mathutils\nfrom math import...,
5,13,126,user,"how to run a 

## Pivot columns for saving

In [27]:

model_columns = ["chatgpt-4o-latest", "o3-2025-04-16", "gpt-5-chat-latest", "gpt-4.1-2025-04-14", "gpt-4o-2024-08-06", "claude-3-7-sonnet-20250219", "claude-sonnet-4-20250514"]

melted = prompts.melt(
    id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id"],
    value_vars=model_columns,
    var_name="model_version",
    value_name="model_reply"
)

code_columns = [f"code_{col}" for col in model_columns if f"code_{col}" in prompts.columns]

if code_columns:
    code_melted = prompts.melt(
        id_vars=["conversation_id", "message_id", "message_text", "gender", "user_id"],
        value_vars=code_columns,
        var_name="model_version_code",
        value_name="code"
    )

    code_melted["model_version"] = code_melted["model_version_code"].str.replace("code_", "")

    melted = melted.merge(
        code_melted[["conversation_id", "message_id", "model_version", "code"]],
        on=["conversation_id", "message_id", "model_version"],
        how="left"
    )
else:

    melted["code"] = None


melted = melted.reset_index(drop=True)
melted["code_reply_id"] = melted.index + 1


final_df = melted[
    ["code_reply_id", "conversation_id", "message_id", "message_text", "gender", "user_id",
     "model_reply", "model_version", "code"]
]


final_df

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,model_version,code
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,To perform **dummy hot encoding** (also called...,chatgpt-4o-latest,import pandas as pd\n\n# Sample DataFrame\ndf ...
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,To encode and compress a JavaScript object int...,chatgpt-4o-latest,"import { compressToEncodedURIComponent, decomp..."
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,Certainly! You can split the `persona` column ...,chatgpt-4o-latest,import pandas as pd\n\n# Sample DataFrame (rep...
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,"That's a great productivity idea, especially f...",chatgpt-4o-latest,import os\n\ndef list_pdfs(directory):\n re...
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,You can achieve this in **Blender (3.x or newe...,chatgpt-4o-latest,import bpy\nimport mathutils\n\n# === CONFIGUR...
...,...,...,...,...,...,...,...,...,...
170,171,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies the `parse_obj` fu...,claude-sonnet-4-20250514,import pytest\nfrom your_module import OdooTic...
171,172,72,1534,can you write me some python code to count the...,Woman (cisgender),11,Here are several ways to count objects in JSON...,claude-sonnet-4-20250514,import json\n\ndef count_json_items(json_data)...
172,173,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge these two dataframes based on th...,claude-sonnet-4-20250514,import pandas as pd\n\n# Merge on 'claim' colu...
173,174,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,To find the best k for k-means clustering on y...,claude-sonnet-4-20250514,


## Save to database

In [28]:
final_df.to_sql("translated_scratch_prompts_with_code", conn, if_exists="replace", index=False)

175